# ZOONIVERSE DROP-DOWN TEXT REDUCER AND CARD COMBINER
This Jupyter notebook contains basic code to process Zooniverse data beyond the Panoptes GUI provided by Zooniverse.

As of the time this was written (December, 2020), the "Data Exports" page on Zooniverse allows you to download a Panoptes Aggregation GUI. This GUI creates a reduced consensus file for both drop-downs and text. However, the reduced drop-down file is currently hashed and not fully reduced. This code attempts to complete those final steps by unhashing the drop-down file and combining the text and drop-downs.

### Inputs
This takes as input the files paths for the csvs...
- workflows (from Zooniverse data export)
- reduced drop-downs (from Panoptes)
- reduced text (from Panoptes)
- where you would like to output an unhashed reduced drop-down csv
- where you would like to output a combined reduced drop-down (unhashed) and free text csv

### Outputs
This outputs csvs...
- "dropdown_max_reduced.csv" unhashed reduced drop-down csv
- "final.csv" combined reduced drop-down (unhashed) and free text csv

### Process
It does so by using the workflows csv to create a nested dictionary (workflow --> task --> hash --> text) to unhash the reduced drop-down text
It then unhashes the reduced drop-down csv and writes this data to a new max reduced file
It then combines the max reduced drop-down text and reduced text into a new file

### Known Code Flaws
Please be advised this code is rustic and contains several potential flaws...
- it has not been factored out into highly re-usable functions
- it does not preserve the order of the headers when it combines the csv files, so the user must re-order them in Excel manually
- it iterates through the csv files and data structures in their entirety (this could become a problem with larger volumes of data)
- it makes some assumptions about the structure of the Zooniverse data such as the fact that the workflows csv is ordered from oldest to newest version, and it hard-codes the header names it uses to access data (ex. "workflow_id")

### Questions?
If you have questions about this code's origin or purpose, please contact Library-RDS@northeastern.edu

For supporting documentation on the Panoptes GUI, please see https://aggregation-caesar.zooniverse.org/index.html

For a friendly "idiot's guide" explanation to the back-end of Zooniverse, please see https://www.zooniverse.org/talk/18/1439900


## STEP 1: Upload Files
There are 3 CSV files that you will need to upload:
- workflows (from Zooniverse data export, ex. "boston-phoenix-1974-workflows.csv")
- reduced drop-downs (from Panoptes GUI output, ex. "dropdown_reducer_reductions.csv")
- reduced text (from Panoptes GUI output, ex. "text_reducer_reductions.csv")

To upload your files, do the following:

<ol>
    <li style="clear:both">Go to the main menu for this project by clicking on the "Jupyter" icon in the upper left-hand corner of your screen.<br></br><img src="https://raw.githubusercontent.com/KNortheastern/Zooniverse/master/logo.png" style="float:left"></li>
    <li style="clear:both">Click the "Upload" button on the right-hand side of the screen and, when prompted, select the file you want to upload.<br></br><img src="https://raw.githubusercontent.com/KNortheastern/Zooniverse/master/Screenshots2.png" style="float:left" width="650"></li>
    <li style="clear:both">When the file is ready to be uploaded, it will appear on the page with another "Upload" button to confirm the upload. Click the "Upload" button to complete the uploading of this file.<br></br><img src="https://raw.githubusercontent.com/KNortheastern/Zooniverse/master/Screenshots4.png" style="float:left" width="650"></li>
    <li style="clear:both">Repeat this upload process for all 3 CSVs. At the end, your screen should look something like this.<br></br><img src="https://raw.githubusercontent.com/KNortheastern/Zooniverse/master/Screenshots6.png" style="float:left" width="650"></li>
</ol>

## STEP 2: Update file names if necessary
Please check the file names in the code cell below. If necessary, update the file names to match the ones that you are using. When finished checking and updating, run the code cell.
- csvPathWorkflows contains the name of the workflows CSV
- csvPathReduced contains the name of the reduced dropdown CSV
- csvTextRed contains the name of the reduced text CSV

In [ ]:
# specify file paths for CSV inputs
csvPathWorkflows = "./boston-phoenix-1974-workflows.csv"
csvPathReduced = "./dropdown_reducer_reductions.csv"
csvTextRed = "./text_reducer_reductions.csv"

---

## STEP 3: Run code
Run each of the code cells in this step, in order. The steps are broken out with short explanations in case changes need to be made in the future or errors occur and there is a need for troubleshooting.
### STEP 3a: set up the task

In [ ]:
# import any additional libraries we need
import csv # to read CSVs
import json # to read JSONs

# specify file paths for CSV outputs
csvPathMaxRed = "./dropdown_max_reduced.csv"
csvFinal = "./final.csv"

### STEP 3b: read the workflows csv file and create a nested dictionary to unhash the drop-down data
Note: the dictionary will be used to look up each hash by workflow --> task --> hash --> text

In [ ]:
# create an empty dictionary to hold the outer level indexed by workflow (workflow --> task --> hash --> text)
workflowDict = {}

# read the workflows csv file and populate the dictionary with an entry for each workflow
with open(csvPathWorkflows, 'r') as csvFile:
    csvReader = csv.DictReader(csvFile)

    # for each row in the csv file...
    for eachRow in csvReader:
        # Note: eachRow is a dict with the following keys (can see using eachRow.keys() )
            #['tutorial_subject_id',
            # 'tasks',
            # 'display_name',
            # 'prioritized',
            # 'first_task',
            # 'aggregation',
            # 'classifications_count',
            # 'minor_version',
            # 'grouped',
            # 'workflow_id',
            # 'version',
            # 'pairwise',
            # 'retirement',
            # 'primary_language',
            # 'active',
            # 'retired_set_member_subjects_count',
            # 'strings']
            
        # create a unique ID for the workflow using "workflow_id"
        # note: the file also contains "version" and "minor_version" data, but this is safely ignored
        #       because the csv is written in order from oldest to newest such that iterating through
        #       the csv file overwrites (and therefore updates) the older entries
        workflowId = eachRow['workflow_id']
        uniqueID = workflowId

        # create an empty dictionary to hold the second level dictionary indexed by task (task --> hash --> text)
        workflowTaskDict = {}

        # grab the tasks and strings columns as JSONs
        tasksDict = json.loads(eachRow['tasks']) # contains hash --> label (ex. "T1.selects.0.options.*.0.label")
        stringsDict = json.loads(eachRow['strings']) # contains label --> text (ex. "none")

        # iterate through the tasks and for each task, populate the dictionary by...
        for key,val in list(tasksDict.items()):
            # create an empty dictionary to hold third level dictionary indexed by hash (hash --> text)
            workflowHashDict = {}

            # try to do the following...
            try:
                # grab the list containing pairs of hashes (ex. "5bd2fbcf507d4") and labels (ex. "T1.selects.0.options.*.0.label")
                hashList = val['selects'][0]['options']['*']

                # iterate through each pair in the hashlist and...
                for eachDict in hashList:

                    # grab the hash and label
                    hashValue = str(eachDict['value'])
                    hashLabel = eachDict['label']

                    # look the label up in the strings dictionary
                    hashString = stringsDict[hashLabel]

                    # create a new dictionary entry for this hash
                    workflowHashDict[hashValue] = hashString

                # create a new dictionary entry in the workflow for this task
                workflowTaskDict[key] = workflowHashDict

            # if an error while trying, do nothing...
            except KeyError:
                continue

        # create a new entry in the overall workflow dictionary for all of this workflow's tasks
        workflowDict[uniqueID] = workflowTaskDict

### STEP 3c: use the hash dictionary to unhash the reduced drop-downs csv file and write a new max reduced file
Note: This step outputs the "dropdown_max_reduced.csv" CSV file.

In [ ]:
# read the reduced dropdowns csv file
with open(csvPathReduced, 'r') as csvFile:
    csvReader = csv.DictReader(csvFile)

    # start the process of writing the new file
    csvHeaders = list(next(csvReader).keys())
    writer = csv.writer(open(csvPathMaxRed, 'w'))
    csvHeaders.append("Unhashed") # add column header for new unhashed data
    writer.writerow(csvHeaders)

    # for each row in the reduced drop-downs csv file...
    for eachRow in csvReader:
        # grab the workflow id, task, and list of hashes
        workflowId = eachRow["workflow_id"]
        taskID = eachRow["task"]
        hashDict = eval(eachRow["data.value"])[0] # ex. "{'None': 1, 'cd638da74b4e3': 1, 'e4d9c81e23fe3': 5}"
        numHash = len(hashDict)

        # assume there is no unhashed answer until you can get an answer
        unHashed = ""

        # for each hash...
        for key,val in list(hashDict.items()):

            # try to look up the hash in the workflow (workflow --> task --> hash --> text) dictionary...
            try:
                # if there is only one hash voted on, make the unhashed answer that answer
                if numHash == 1:
                    unHashed = str(workflowDict[workflowId][taskID][key])
                # otherwise, make a long string with all of the answers and the votes they received
                else:
                    unHashed = unHashed + " [" + str(val) + " votes] " + str(workflowDict[workflowId][taskID][key])

            # if the hash wasn't in the dictionary, handle it as free-text...
            except KeyError:
                # if there is only one free-text voted on, make the unhashed answer that answer
                if numHash == 1:
                    unHashed = str(key)
                # otherwise, add the free text-to the string of answers along with the number of votes it received
                else:
                    unHashed = unHashed + " [" + str(val) + " votes] " + str(key)
                continue
                
        # add the unhashed data to this row as though it had been there all along
        eachRow["Unhashed"] = unHashed

        # write the row (now with unhashed data) to the new file
        writer.writerow(tuple(eachRow.values()))

### STEP 3d: combine the new max reduced drop-down file with the reduced text file
Note: This step outputs the "final.csv" CSV file.

In [ ]:
# figure out combined headers for the new combined csv file
# assume there are no headers at first
combinedHeaders = []


# find the headers from the max reduced drop-down csv file
with open(csvPathMaxRed, 'r') as csvFile:
    csvReader = csv.DictReader(csvFile)
    csvPathMaxRedHeaders = list(next(csvReader).keys())

# find the headers from the reduced text csv file
with open(csvTextRed, 'r') as csvFile:
    csvReader = csv.DictReader(csvFile)
    csvTextRedHeaders = list(next(csvReader).keys())

# combine the two sets of headers
combinedHeaders = list(set(csvPathMaxRedHeaders + csvTextRedHeaders))


# figure out combined data for the new combined csv file
# add the data from the max reduced drop-down csv file
csvPathMaxRedData = []
with open(csvPathMaxRed, 'r') as csvFile:
    csvReader = csv.DictReader(csvFile)

    for eachRow in csvReader:

        # create a blank dictionary with all of the combined headers
        rowOfDataDict = {}
        for eachKey in combinedHeaders:
            rowOfDataDict[eachKey] = ""

        # populate as much of this dictionary as we can (i.e. some headers won't be relevant to this csv)
        for key,val in list(eachRow.items()):
            rowOfDataDict[key] = val

        csvPathMaxRedData.append(rowOfDataDict)

# add the data from the reduced text csv file
csvTextRedData = []
with open(csvTextRed, 'r') as csvFile:
    csvReader = csv.DictReader(csvFile)
    for eachRow in csvReader:

        # create a blank dictionary with all of the combined headers
        rowOfDataDict = {}
        for eachKey in combinedHeaders:
            rowOfDataDict[eachKey] = ""

        # populate as much of this dictionary as we can (i.e. some headers won't be relevant to this csv)
        for key,val in list(eachRow.items()):
            rowOfDataDict[key] = val

        csvTextRedData.append(rowOfDataDict)

# combine the data from both csvs
combinedData = csvPathMaxRedData + csvTextRedData


# write the combined data to the new combined csv file
with open(csvFinal, 'w') as csvfile:  
        writer = csv.DictWriter(csvfile, fieldnames=combinedHeaders)
        writer.writeheader()
        for data in csvPathMaxRedData:
            writer.writerow(data)

        for data in csvTextRedData:
            writer.writerow(data)

## STEP 4: Download output
Once the code has finished running, you should see two new CSV files in your main menu:
- "dropdown_max_reduced.csv" is an intermediate CSV file and contains the unhashed drop-downs
- "final.csv" is the final CSV file and contains the fully reduced and combined data for both the text and drop-downs

Download the "final.csv" file.

<img src="https://raw.githubusercontent.com/KNortheastern/Zooniverse/master/Screenshots7.png" align="left" width="650">
